In [20]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as func
from pathlib import Path
from collections import defaultdict

spark = (
    SparkSession.builder.master("local").appName("Do It Youself - One").getOrCreate()
)

path = Path().home() / "Documents/PySparkCurso/download/Atividades"

In [15]:
def gen_vars(path: Path, session: SparkSession) -> defaultdict[str, DataFrame]:
    vars = defaultdict(
        DataFrame,
    )
    for file in path.iterdir():
        vars[file.name.split(".")[0].lower()] = spark.read.load(
            str(file), header=True, format="parquet", inferSchema=True
        )
    return vars


vars = gen_vars(path, spark)
vars

defaultdict(pyspark.sql.dataframe.DataFrame,
            {'itensvendas': DataFrame[ProdutoID: bigint, VendasID: bigint, Quantidade: bigint, ValorUnitario: double, ValorTotal: double, Desconto: string, TotalComDesconto: double],
             'vendas': DataFrame[VendasID: bigint, VendedorID: bigint, ClienteID: bigint, Data: string, Total: double],
             'clientes': DataFrame[ClienteID: bigint, Cliente: string, Estado: string, Genero: string, Status: string],
             'vendedores': DataFrame[VendedorID: bigint, Vendedor: string],
             'produtos': DataFrame[ProdutoID: bigint, Produto: string, Preco: string]})

In [22]:
vars["clientes"].select(
    func.col("Cliente").alias("Nome"),
    "Estado",
    "Status",
).show()

+--------------------+------+--------+
|                Nome|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [24]:
vars["clientes"].select(func.col("Cliente").alias("Nome"), "Status").where(
    func.col("Status") != "Silver"
).show()

+-------------------+--------+
|               Nome|  Status|
+-------------------+--------+
|   Adriana Guedelha|Platinum|
|   Alberto Monsanto|    Gold|
|      Anna Carvajal|    Gold|
|      Bento Quintão|    Gold|
|      Carminda Dias|    Gold|
|      Cláudio Jorge|    Gold|
|    Dionísio Saltão|    Gold|
|   Firmino Meireles|    Gold|
|      Flor Vilanova|Platinum|
|Honorina Villaverde|    Gold|
|    Ibijara Botelho|Platinum|
|  Iracema Rodríguez|    Gold|
|         Joana Ataí|Platinum|
+-------------------+--------+



 ### Join clientes in vendas

In [37]:
vendas_clientes = (
    vars["vendas"]
    .join(
        vars["clientes"],
        "ClienteID",
        "inner",
    )
    .select("VendasID", func.col("Cliente").alias("Nome"), "Status")
)
vendas_clientes.show()

+--------+--------------------+------+
|VendasID|                Nome|Status|
+--------+--------------------+------+
|     382|Adelina Buenaventura|Silver|
|     254|        Adelino Gago|Silver|
|     336|     Adolfo Patrício|Silver|
|     374|       Adélio Lisboa|Silver|
|     355|       Adélio Lisboa|Silver|
|     292|       Adélio Lisboa|Silver|
|     378|   Alarico Quinterno|Silver|
|     316|        Aldo Martins|Silver|
|     258|        Aldo Martins|Silver|
|     154|   Alexandra Tabares|Silver|
|     147|   Alexandra Tabares|Silver|
|     136|   Alexandra Tabares|Silver|
|     149|      Alfredo Cotrim|Silver|
|      91|      Alfredo Cotrim|Silver|
|     351|     Almeno Figueira|Silver|
|     312|     Amadeu Martinho|Silver|
|     331|           Ana Homem|Silver|
|     313|   Anacleto Guterres|Silver|
|     204|           Anna Beça|Silver|
|     108|           Anna Beça|Silver|
+--------+--------------------+------+
only showing top 20 rows



### Join vendas_clientes on itens venda

In [47]:
status_vendas = vendas_clientes.join(vars["itensvendas"], "VendasID", "inner").select(
    "Status", "Quantidade", func.col("ValorTotal").alias("Total")
)
status_vendas.show()

+------+----------+-------+
|Status|Quantidade|  Total|
+------+----------+-------+
|Silver|         1|7820.85|
|Silver|         1|  97.75|
|Silver|         1|  135.0|
|Silver|         1|  150.4|
|Silver|         1|  155.0|
|Silver|         2| 5932.0|
|Silver|         1|7820.85|
|Silver|         1|  97.75|
|Silver|         2| 5910.0|
|Silver|         1| 2955.0|
|Silver|         1| 7658.0|
|Silver|         1| 8852.0|
|Silver|         1|  121.5|
|Silver|         2|6510.16|
|Silver|         2|18402.0|
|Silver|         1| 7524.2|
|Silver|         2|  376.0|
|Silver|         1|3616.75|
|Silver|         1|  108.0|
|Silver|         1|  115.0|
+------+----------+-------+
only showing top 20 rows



In [49]:
status_vendas.groupBy("Status").agg(
    func.sum("Quantidade").alias("Total Vendas"),
    func.round(func.sum("Total")).alias("Total"),
).show()

+--------+------------+---------+
|  Status|Total Vendas|    Total|
+--------+------------+---------+
|Platinum|           7|  12585.0|
|  Silver|        1046|3014291.0|
|    Gold|           7|  27287.0|
+--------+------------+---------+

